In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import statsmodels.api as sm
import statsmodels.formula.api as smf

#### Import my libraries

In [2]:
my_libs_dir = '../'
if not my_libs_dir in sys.path:
    sys.path.append(my_libs_dir)  # add the path to my_lib directory 

# The following lines are needed to auto-reload my library file
# Without these lines, my library file is read only once and
# modifications of my library file are not reflected.
%load_ext autoreload
%autoreload 1
%aimport my_libs.linear_reg
# import from my library file
from my_libs.linear_reg import step_aic_forward, calc_vifs

#### Step 1. Collect possible explanatory variables
目的変数に影響を与えていそうな要因は、可能な限り網羅的に説明変数に取り入れる。

In [3]:
csv_in = 'google_calendar_survey.csv'
df_all = pd.read_csv(csv_in)
print(df_all.shape)
print(df_all.info())
display(df_all.head())

(12, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   priority              12 non-null     object
 1   layout                12 non-null     int64 
 2   font                  12 non-null     int64 
 3   color                 12 non-null     int64 
 4   duplicate             12 non-null     int64 
 5   create                12 non-null     int64 
 6   edit                  12 non-null     int64 
 7   delete                12 non-null     int64 
 8   view_day_week_month   12 non-null     int64 
 9   change_view_settings  12 non-null     int64 
 10  repeat_schedule       12 non-null     int64 
 11  create-manage_tasks   12 non-null     int64 
 12  share                 12 non-null     int64 
 13  satisfaction          12 non-null     int64 
 14  recommend             12 non-null     int64 
 15  continue_using        12 non-null

,priority,layout,font,color,duplicate,create,edit,delete,view_day_week_month,change_view_settings,repeat_schedule,create-manage_tasks,share,satisfaction,recommend,continue_using
0,usability,5,4,5,4,5,5,5,4,5,5,4,5,5,5,5
1,visibility,4,3,5,5,3,4,5,3,2,5,5,4,4,4,2
2,visibility,5,5,5,4,4,5,5,4,4,5,4,4,4,4,5
3,functionality,4,3,3,3,3,1,3,2,1,2,3,4,3,4,5
4,usability,5,5,5,5,5,5,5,5,5,5,5,5,4,3,2


##### Check numerical / category variables if needed
数値列・カテゴリー列の様子をみる

In [4]:
display(df_all.describe())
display(df_all.describe(exclude='number'))

,layout,font,color,duplicate,create,edit,delete,view_day_week_month,change_view_settings,repeat_schedule,create-manage_tasks,share,satisfaction,recommend,continue_using
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,4.333333,3.916667,4.250000,4.166667,3.833333,3.916667,4.416667,3.583333,3.333333,4.000000,3.916667,3.833333,3.666667,3.250000,3.083333
std,0.651339,0.668558,0.753778,0.717741,1.029857,1.378954,0.900337,1.164500,1.154701,1.044466,0.792961,1.114641,0.887625,1.288057,1.781640
min,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,3.000000,2.000000,1.000000,2.000000,3.000000,2.000000,2.000000,1.000000,1.000000
25%,4.000000,3.750000,4.000000,4.000000,3.000000,3.000000,3.750000,2.750000,3.000000,3.000000,3.000000,3.000000,3.750000,2.750000,1.750000
50%,4.000000,4.000000,4.000000,4.000000,4.000000,4.500000,5.000000,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,2.500000
75%,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.250000,4.000000,5.000000,4.250000,5.000000,4.000000,4.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


,priority
count,12
unique,3
top,visibility
freq,6


##### Separate explanatory variables and objective variable
説明変数と目的変数を分ける

In [5]:
X = df_all.loc[:, 'layout':'share']  # explanatory variables
y = df_all['continue_using']  # objective variable
print('X:', X.shape)
display(X.head())
print('y:', y.shape)
print(y.head())

X: (12, 12)


,layout,font,color,duplicate,create,edit,delete,view_day_week_month,change_view_settings,repeat_schedule,create-manage_tasks,share
0,5,4,5,4,5,5,5,4,5,5,4,5
1,4,3,5,5,3,4,5,3,2,5,5,4
2,5,5,5,4,4,5,5,4,4,5,4,4
3,4,3,3,3,3,1,3,2,1,2,3,4
4,5,5,5,5,5,5,5,5,5,5,5,5


y: (12,)
0    5
1    2
2    5
3    5
4    2
Name: continue_using, dtype: int64


#### Step 2. Correlation coefficients between all combination of explanatory variables
変数間の相関係数を算出。

##### all by all Pearson correlation coefficients;
総当たりのPearson相関係数

In [6]:
corr_all = X.corr(method='pearson')
display(corr_all)

,layout,font,color,duplicate,create,edit,delete,view_day_week_month,change_view_settings,repeat_schedule,create-manage_tasks,share
layout,1.000000,0.695889,0.555492,0.453743,0.632456,0.641036,0.516742,0.559329,0.564076,0.534522,0.410700,0.333914
font,0.695889,1.000000,0.586284,0.221028,0.506137,0.682048,0.516019,0.651962,0.745815,0.520756,0.328672,0.101660
color,0.555492,0.586284,1.000000,0.420084,0.292770,0.634091,0.770241,0.440162,0.522233,0.692820,0.494305,0.054100
duplicate,0.453743,0.221028,0.420084,1.000000,0.286972,0.382718,0.304808,0.199408,0.365636,0.485071,0.505813,0.265144
create,0.632456,0.506137,0.292770,0.286972,1.000000,0.757508,0.669974,0.164241,0.509647,0.169031,-0.018554,0.052796
edit,0.641036,0.682048,0.634091,0.382718,0.757508,1.000000,0.835972,0.542543,0.647062,0.631194,0.408767,0.049288
delete,0.516742,0.516019,0.770241,0.304808,0.669974,0.835972,1.000000,0.267352,0.378927,0.386695,0.180392,-0.196273
view_day_week_month,0.559329,0.651962,0.440162,0.199408,0.164241,0.542543,0.267352,1.000000,0.653544,0.747435,0.746579,0.431900
change_view_settings,0.564076,0.745815,0.522233,0.365636,0.509647,0.647062,0.378927,0.653544,1.000000,0.678401,0.430237,0.470882
repeat_schedule,0.534522,0.520756,0.692820,0.485071,0.169031,0.631194,0.386695,0.747435,0.678401,1.000000,0.878114,0.546608


In [25]:
print(df_all['priority'].value_counts())

priority
visibility       6
usability        5
functionality    1
Name: count, dtype: int64


##### Pickup explanatory variable pairs with large absolute value of correlation coefficient;
相関係数の絶対値が大きい説明変数ペアの出力

In [7]:
th_corr = 0.3
keep = np.triu(np.ones(corr_all.shape), k=1).astype('bool').flatten()
#print(np.ones(corr_all.shape))  # debug
#print(np.triu(np.ones(corr_all.shape), k=1))  # debug
#print(np.triu(np.ones(corr_all.shape), k=1).astype('bool'))  # debug
#print(keep)  # debug
triu = corr_all.stack()[keep]
#print(corr_all.stack())  # debug
triu_sorted = triu[ np.abs(triu).sort_values(ascending=False).index ]
#print(np.abs(triu).sort_values(ascending=False))  # debug
#print(np.abs(triu).sort_values(ascending=False).index)  # debug
print(triu_sorted[ (triu_sorted < -th_corr) | (triu_sorted > th_corr) ])

repeat_schedule       create-manage_tasks     0.878114
edit                  delete                  0.835972
color                 delete                  0.770241
create                edit                    0.757508
view_day_week_month   repeat_schedule         0.747435
                      create-manage_tasks     0.746579
font                  change_view_settings    0.745815
layout                font                    0.695889
color                 repeat_schedule         0.692820
font                  edit                    0.682048
change_view_settings  repeat_schedule         0.678401
create                delete                  0.669974
view_day_week_month   change_view_settings    0.653544
font                  view_day_week_month     0.651962
edit                  change_view_settings    0.647062
layout                edit                    0.641036
color                 edit                    0.634091
layout                create                  0.632456
edit      

#### Step 3. MLR calculation using all variables
全説明変数を用いて、標準化なしで線形重回帰分析

In [8]:
X_c = sm.add_constant(X)
model = sm.OLS(y, X_c)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         continue_using   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 27 Jun 2024   Prob (F-statistic):                nan
Time:                        09:13:49   Log-Likelihood:                 355.12
No. Observations:                  12   AIC:                            -686.2
Df Residuals:                       0   BIC:                            -680.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    5.9330 

C:\Users\iniad\anaconda3\envs\ds2024\lib\site-packages\scipy\stats\_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  k, _ = kurtosistest(a, axis)
C:\Users\iniad\anaconda3\envs\ds2024\lib\site-packages\statsmodels\regression\linear_model.py:1795: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\iniad\anaconda3\envs\ds2024\lib\site-packages\statsmodels\regression\linear_model.py:1795: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\iniad\anaconda3\envs\ds2024\lib\site-packages\statsmodels\regression\linear_model.py:1717: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


#### Step 4. Check R2 and Adjusted R2 to see whether MLR is appropriate for this data
決定係数や自由度調整済み決定係数をみて、そもそも線形モデルの当てはめが妥当かどうかを判断

In [9]:
print('R2:', results.rsquared)
print('Adj R2:', results.rsquared_adj)

R2: 1.0
Adj R2: nan


In [10]:
print(results.params)

const                   5.933050
layout                  2.155593
font                   -0.024424
color                  -0.942305
duplicate              -3.176123
create                  1.209496
edit                   -0.381827
delete                 -0.412469
view_day_week_month    -1.402357
change_view_settings   -0.052639
repeat_schedule         1.818821
create-manage_tasks     1.602903
share                  -1.181281
dtype: float64


In [11]:
print(results.params.sort_values(key=np.abs, ascending=False))

const                   5.933050
duplicate              -3.176123
layout                  2.155593
repeat_schedule         1.818821
create-manage_tasks     1.602903
view_day_week_month    -1.402357
create                  1.209496
share                  -1.181281
color                  -0.942305
delete                 -0.412469
edit                   -0.381827
change_view_settings   -0.052639
font                   -0.024424
dtype: float64


#### Step 6. Standardization of variables
Compare coefficients for explanatory variables  
全説明変数と目的変数を標準化して線形重回帰分析  
標準化偏回帰係数を比較

In [12]:
# NOTE: after scaling, X_scaled and Y_scaled are ndarray, not DataFrame.
X_scaled = preprocessing.scale(X)
y_scaled = preprocessing.scale(y)
# So, make DataFrames corresponding to X_scaled and y_scaled.
dfX_scaled = pd.DataFrame(X_scaled, columns=X.columns)
sery_scaled = pd.Series(y_scaled, name=y.name)

model = sm.OLS(sery_scaled, dfX_scaled)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:         continue_using   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          3.582e+28
Date:                Thu, 27 Jun 2024   Prob (F-statistic):                    4.12e-15
Time:                        09:13:49   Log-Likelihood:                          391.85
No. Observations:                  12   AIC:                                     -761.7
Df Residuals:                       1   BIC:                                     -756.4
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

C:\Users\iniad\anaconda3\envs\ds2024\lib\site-packages\scipy\stats\_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  k, _ = kurtosistest(a, axis)


In [13]:
print(results.params.sort_values(key=np.abs, ascending=False))

duplicate              -1.168753
delete                 -0.794864
create                  0.779630
repeat_schedule         0.772511
view_day_week_month    -0.660655
layout                  0.568313
share                  -0.473208
create-manage_tasks     0.408814
change_view_settings   -0.388184
edit                    0.259957
color                   0.160779
font                   -0.002466
dtype: float64


#### Step 7. Feature selection
変数選択

In [14]:
exog = list(dfX_scaled.columns)  # Initial set = all explanatory variables
endog = [sery_scaled.name]  # Objective variables
df_scaled = pd.concat([dfX_scaled, sery_scaled], axis=1)

#### by forward selection method based on AIC
変数増加法による変数選択

In [15]:
results_aic=step_aic_forward(smf.ols, exog, endog, data=df_scaled)

AIC: 36.055, formula: continue_using ~ 1
AIC: 37.519, formula: continue_using ~ layout
AIC: 37.795, formula: continue_using ~ delete
AIC: 38.026, formula: continue_using ~ repeat_schedule
AIC: 38.047, formula: continue_using ~ view_day_week_month
AIC: 36.879, formula: continue_using ~ create-manage_tasks
AIC: 33.121, formula: continue_using ~ duplicate
AIC: 38.051, formula: continue_using ~ color
AIC: 37.747, formula: continue_using ~ font
AIC: 37.679, formula: continue_using ~ share
AIC: 37.431, formula: continue_using ~ edit
AIC: 36.879, formula: continue_using ~ create
AIC: 37.926, formula: continue_using ~ change_view_settings
AIC: 28.506, formula: continue_using ~ duplicate + layout
AIC: 32.829, formula: continue_using ~ duplicate + delete
AIC: 33.765, formula: continue_using ~ duplicate + repeat_schedule
AIC: 34.967, formula: continue_using ~ duplicate + view_day_week_month
AIC: 29.644, formula: continue_using ~ duplicate + create-manage_tasks
AIC: 33.932, formula: continue_using

In [16]:
print(results_aic.aic)
print(results_aic.model.exog_names)
print(results_aic.model.endog_names)

28.50573043786668
['Intercept', 'duplicate', 'layout']
continue_using


#### Step 8. Check of multicolinearity (VIF checkup)
マルチコのチェック

・Iteration of Variable selection (selected_cols) <-> Check VIF
until all VIFs < 10.  
・NOTE: standardization of variables does not affect the results

In [17]:
endogs = results_aic.model.endog_names
exogs = results_aic.model.exog_names.copy()
exogs.remove('Intercept')
print(exogs)  # debug
X_c = sm.add_constant(X[exogs])
vifs = calc_vifs(X_c, y)
display(vifs)

['duplicate', 'layout']


,VIF
const,59.952381
duplicate,1.259259
layout,1.259259


For all explantory variables, VIF < 10, so we can go forward.

#### Step 9. Estimate the magnitude of the influence of each explanatory variable on the objective variable
最終的に得られた標準化偏回帰係数から、各説明変数の目的変数に対する影響の大きさがわかる

In [19]:
X_final_scaled = dfX_scaled[exogs]
model_final_scaled = sm.OLS(y_scaled, X_final_scaled)
results_final_scaled = model_final_scaled.fit()
print(results_final_scaled.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.618
Model:                            OLS   Adj. R-squared (uncentered):              0.542
Method:                 Least Squares   F-statistic:                              8.090
Date:                Thu, 27 Jun 2024   Prob (F-statistic):                     0.00813
Time:                        09:14:56   Log-Likelihood:                         -11.253
No. Observations:                  12   AIC:                                      26.51
Df Residuals:                      10   BIC:                                      27.48
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

C:\Users\iniad\anaconda3\envs\ds2024\lib\site-packages\scipy\stats\_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  k, _ = kurtosistest(a, axis)


In [20]:
print(results_final_scaled.params.sort_values(key=np.abs, ascending=False))

duplicate   -0.850469
layout       0.594799
dtype: float64


The order of strengths of influences on objective variables  
（これが、目的変数に対する各説明変数の影響の大きさ順）

#### Step 10. Stat. test for MLR equation
重回帰式の検定 (求めた重回帰式は目的変数を説明している？)

In [21]:
print('p-values (F-statistic)', results_final_scaled.f_pvalue)

p-values (F-statistic) 0.008131599375615845


#### Step 11. MLR calculation using selected explanatory variables
選択された説明変数を用いて、標準化なしで線形重回帰分析

In [22]:
X_final_c = sm.add_constant(X[exogs])
model_final = sm.OLS(y, X_final_c)
results_final = model_final.fit()
print(results_final.summary())

                            OLS Regression Results                            
Dep. Variable:         continue_using   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     7.281
Date:                Thu, 27 Jun 2024   Prob (F-statistic):             0.0132
Time:                        09:14:56   Log-Likelihood:                -17.661
No. Observations:                  12   AIC:                             41.32
Df Residuals:                       9   BIC:                             42.78
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.8294      2.721      1.775      0.1

C:\Users\iniad\anaconda3\envs\ds2024\lib\site-packages\scipy\stats\_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  k, _ = kurtosistest(a, axis)


#### Step 12. Check R2 and Adjusted R2
決定係数や自由度調整済み決定係数をみて、線形モデルの当てはまりの良さをチェック

In [23]:
print('R2:', results_final.rsquared)
print('Adj R2:', results_final.rsquared_adj)

R2: 0.6180247755426753
Adj R2: 0.5331413923299364


#### Step 13. partial regression coefficients
最終的に得られた偏回帰係数から、「各説明変数が1増えたときの目的変数の増分」がわかる。

In [24]:
print(results_final.params)

const        4.829365
duplicate   -2.111111
layout       1.626984
dtype: float64


Coefficients of MLR;  
Increment of objective variable when corresponding variable is increased by 1
and other variables are not changed